In [32]:
% matplotlib inline
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
from operator import add

In [55]:
# Input of the name of the lattice and the temperature you wish to study
filename1 = 'shakti600/shakti600_'
filename2 = '#_XMCD_6_13.csv'
temperature = 218
# Input coordinates of the unit cell of the cluster in forms of [row,column]
initinput = [[2,2],[2,3],[3,3],[3,4],[4,3],[4,4]]
# Input the unit vector of the lattice
v1 = [0,4]
v2 = [4,0]

In [56]:
# This cell use the bubble sort to sort the index, enabling coding 
for i in range(len(init)-1,0,-1):
    for j in range(i,0,-1):
        if(init[j]<init[j-1]):
            temp = init[j]
            init[j] = init[j-1]
            init[j-1] = temp
init

[[4, 4], [4, 5], [5, 5], [5, 6], [6, 5], [6, 6]]

In [57]:
# Create an asymmetric matrix to record the configuration transformation. Each row represents the starting configuration
# while the columns represents the end configuration. The diagonal represents a static configuration.
len_unitcell = len(init)
matrixdim = pow(2,len_unitcell)

In [59]:
matrix = np.zeros([matrixdim,matrixdim])
for i in range(1,5):
    filename = filename1+str(temperature)+'K_'+str(i)+filename2
    if(os.path.isfile(filename)):
        # Read the file
        array = pd.read_csv(filename,index_col=(0,1),header =None)
        if(array.index[2]==(1,3)):
            print('this is shakti1')
            init = [[i[0]+2,i[1]+2] for i in initinput]
        elif(array.index[2]==(1,4)):
            print('this is shakti')
            init = initinput
        else :
            print('erro, please check the file:')
            print(filename)
        # get the maximum row, column and time number
        (rowmax,columnax) = max(array.index)
        timemax = max(array.columns) # time runs from 2 to timemax
        for m in range(-10,10):
            for n in range(-10,10):
                unitcell = []
                for initcell  in init:
                    #print([a + b +c for a, b, c in zip(initcell, v1, v2)])
                    unitcell.append([a + b*m +c*n for a, b, c in zip(initcell, v1, v2)])
                #print(unitcell)
                rowhigh = max(max(p[0:1]) for p in unitcell)
                rowlow  = min(min(p[0:1]) for p in unitcell)
                columnhigh = max(max(p[1:]) for p in unitcell)
                columnlow = min(min(p[1:]) for p in unitcell)
                if(rowlow>0 and columnlow>0 and rowhigh <=rowmax and columnhigh<=columnax):
                    print(unitcell)
                    for j in range(2,timemax):
                        start_encode = 0
                        end_encode = 0
                        for k in range(len_unitcell):
                            #print(unitcell[k][0],unitcell[k][1])
                            start_encode = start_encode + pow(2,k)*(array.loc[unitcell[k][0],unitcell[k][1]][j]+1)/2
                            end_encode = end_encode +  pow(2,k)*(array.loc[unitcell[k][0],unitcell[k][1]][j+1]+1)/2
                        if(start_encode%1 == 0 and end_encode%1 == 0):
                            #print(start_encode,end_encode)
                            matrix[int(start_encode),int(end_encode)] = matrix[int(start_encode),int(end_encode)] + 1
matrix

this is shakti
[[2, 2], [2, 3], [3, 3], [3, 4], [4, 3], [4, 4]]
[[6, 2], [6, 3], [7, 3], [7, 4], [8, 3], [8, 4]]
[[10, 2], [10, 3], [11, 3], [11, 4], [12, 3], [12, 4]]
[[14, 2], [14, 3], [15, 3], [15, 4], [16, 3], [16, 4]]
[[18, 2], [18, 3], [19, 3], [19, 4], [20, 3], [20, 4]]
[[22, 2], [22, 3], [23, 3], [23, 4], [24, 3], [24, 4]]
[[26, 2], [26, 3], [27, 3], [27, 4], [28, 3], [28, 4]]
[[2, 6], [2, 7], [3, 7], [3, 8], [4, 7], [4, 8]]
[[6, 6], [6, 7], [7, 7], [7, 8], [8, 7], [8, 8]]
[[10, 6], [10, 7], [11, 7], [11, 8], [12, 7], [12, 8]]
[[14, 6], [14, 7], [15, 7], [15, 8], [16, 7], [16, 8]]
[[18, 6], [18, 7], [19, 7], [19, 8], [20, 7], [20, 8]]
[[22, 6], [22, 7], [23, 7], [23, 8], [24, 7], [24, 8]]
[[26, 6], [26, 7], [27, 7], [27, 8], [28, 7], [28, 8]]
[[2, 10], [2, 11], [3, 11], [3, 12], [4, 11], [4, 12]]
[[6, 10], [6, 11], [7, 11], [7, 12], [8, 11], [8, 12]]
[[10, 10], [10, 11], [11, 11], [11, 12], [12, 11], [12, 12]]
[[14, 10], [14, 11], [15, 11], [15, 12], [16, 11], [16, 12]]
[[18, 1

array([[ 53.,   2.,  16., ...,   0.,   0.,   0.],
       [  1.,   2.,   1., ...,   0.,   0.,   0.],
       [ 16.,   0.,  69., ...,   0.,   0.,   0.],
       ..., 
       [  0.,   0.,   0., ...,  57.,   0.,   6.],
       [  0.,   0.,   0., ...,   0.,   1.,   1.],
       [  0.,   0.,   0., ...,   8.,   1.,  46.]])

In [60]:
# Convert the adjacency matrix into a graph. The value of nodesize represent the number of static frames 
# for that configuration. The weight of the edges represents the number of transformation.
matrixnodiag = matrix.copy()
for i in range(matrixdim):
    matrixnodiag[i,i]=0
print(matrixnodiag)
G=nx.from_numpy_matrix(matrixnodiag,create_using=nx.MultiDiGraph())
for i in range(matrixdim):
    G.node[i]['nodesize'] = int(sum(matrix[:,i]))
#G.node[1]
#plt.figure(figsize=(20,10))
#nx.draw_networkx(G)
#plt.show()

[[  0.   2.  16. ...,   0.   0.   0.]
 [  1.   0.   1. ...,   0.   0.   0.]
 [ 16.   0.   0. ...,   0.   0.   0.]
 ..., 
 [  0.   0.   0. ...,   0.   0.   6.]
 [  0.   0.   0. ...,   0.   0.   1.]
 [  0.   0.   0. ...,   8.   1.   0.]]


In [63]:
nx.write_graphml(G,'configuration_transformation/shakti'+str(temperature)+'K.graphml')

In [62]:
sum(sum(matrix[:,:]))

13671.0